In [ ]:
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Activation, Embedding, Conv1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt


from dataprocessing.find_top_movie_with_sequence import find_top_dataset
from dataprocessing.label_and_feature import getSortedClassContents, checkConsists, buildFeature, saveClassesToFile
from dataprocessing.time import checkingTimeDifferent
from recom_model import trainWithBidirectional,trainWithEmbeddingDense

In [ ]:
MIN_CONTENTS_ON_USER = 250
MAX_DAYS = 7
MAX_SEQUENCE = 6
DATA_DIR = "data"

recommandation_df = pd.read_csv('{}/normalizedata.csv'.format(DATA_DIR)).sort_values(by=['time'])

In [ ]:
sortedClassContents = getSortedClassContents(recommandation_df, MIN_CONTENTS_ON_USER)
saveClassesToFile(sortedClassContents)
lenSortedClassContents = len(sortedClassContents)
print(lenSortedClassContents)

In [ ]:
visitors_df = recommandation_df['visitor'].drop_duplicates()
maxItem = recommandation_df['Content'].max()
training_data = []
for index, item in visitors_df.iteritems():
    video = recommandation_df[recommandation_df['visitor'] == item]
    if video.size > 1:
        tempContents = []
        indexContent = 0
        for index, item in video['Content'].iteritems():
            if item not in sortedClassContents:
                continue
            if len(tempContents) > MAX_SEQUENCE:
                tempContents = tempContents[1:]
                continue
            if checkConsists(item, tempContents):
                continue
            if len(tempContents) > 0:
                nowDate = video['time'].iloc[indexContent]
                beforeDate = video['time'].iloc[indexContent - 1]

                if checkingTimeDifferent(nowDate, beforeDate, MAX_DAYS):
                    tempContents = []
                else:
                    feature = buildFeature(tempContents, sortedClassContents, MAX_SEQUENCE)
                    label = sortedClassContents.index(item)
                    training_data.append([[feature], label])
            tempContents.append(item)
            indexContent += 1

training_data = find_top_dataset(training_data)
print("training size: ", len(training_data))
random.shuffle(training_data)

In [ ]:
features = []
labels = []
for feature, label in training_data:
    features.append(feature)
    labels.append(label)

In [ ]:
X = np.array(features)
Y = np.array(labels)
print(X.shape)



In [ ]:
# for lstm with embedding
X = np.squeeze(X, axis=1)
model = trainWithBidirectional(MAX_SEQUENCE, lenSortedClassContents)

In [ ]:
# for pure lstm

X = np.squeeze(X, axis=1)
X = X.reshape(X.shape[0], X.shape[1], 1)
model = Sequential()
model.add(LSTM(256, input_shape=X.shape[1:], return_sequences=True) )
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dense(lenSortedClassContents))
model.add(Activation("softmax"))
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
# for dense
X = np.squeeze(X, axis=1)
print(X.shape)
model = Sequential()   
model.add(Dense(512, activation="relu", input_shape=(X.shape[1:])))
model.add(Dense(512, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(lenSortedClassContents))
model.add(Activation("softmax"))
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [ ]:
# for embedding Dense
X = np.squeeze(X, axis=1)
model = trainWithEmbeddingDense(MAX_SEQUENCE, lenSortedClassContents)

In [ ]:
name = "testing-1"
tensorboard = TensorBoard(log_dir='logs/{}'.format(name))
history = model.fit(X, Y, epochs=250, batch_size=64, verbose=1, callbacks=[tensorboard])

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


In [ ]:
plot_graphs(history, 'loss')

In [ ]:
plot_graphs(history, 'val_loss')

In [ ]:
plot_graphs(history, 'val_accuracy')

In [ ]:
def get5TopPredict(predict):
    predicts = predict[0].argsort()[-5:][::-1]
    for i in predicts:
        print(sortedClassContents[i])
        
def predict(first, second, third, forth, fifth, sixth):
    predict = model.predict([[[sortedClassContents.index(first)],[sortedClassContents.index(second)],[sortedClassContents.index(third)],[sortedClassContents.index(forth)], [sortedClassContents.index(fifth)],[sortedClassContents.index(sixth)]]])
    get5TopPredict(predict)
    return sortedClassContents[np.argmax(predict[0])]


In [ ]:
predictResult = predictDense(0,0,0,0,700,1756)
print(predictResult)

In [ ]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1,6], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "model"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)


In [ ]:
MODEL_DIR = "model"
model.save(MODEL_DIR) 

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

In [ ]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)
print(input_details[0]["index"])
for i in range(TEST_CASES):
  expected = model.predict([[0,0,0,i]])
  interpreter.set_tensor( input_details[0]["index"], np.array([[0,0,0,i]]).astype(np.float32))
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()


In [ ]:
open("tf_model/converted_model.tflite", "wb").write(tflite_model)

In [ ]:
interpreter.set_tensor( input_details[0]["index"], np.array([[0,0,0,sortedClassContents.index(700)]]).astype(np.float32))
interpreter.invoke()
result = interpreter.get_tensor(output_details[0]["index"])
print(sortedClassContents[np.argmax(result)])